In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config

import twelvedata
from twelvedata import TDClient


In [2]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [3]:
# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()


In [4]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()
start_date = end_date - timedelta(days=420)  

# Twelve Data API parameters
symbol = "XAU/USD"
interval = "1min"
outputsize = 5000  
parent_df = pd.DataFrame()  # Initialize an empty parent DataFrame

# Increment through the time range in start_date and end_date by 5 days every 30 seconds
while start_date < end_date:
    time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=(start_date + timedelta(days=5)).strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
    
    # Convert to pandas Dataframes
    time_series_df = time_series.as_pandas()
    time_series_df = time_series_df.astype(str)
    
    # Concatenate the current time series DataFrame to the parent DataFrame
    parent_df = pd.concat([parent_df, time_series_df], ignore_index=False)
 
    # Increment the start_date
    start_date += timedelta(days=5)
    
    # Sleep for 30 seconds
    time.sleep(6)
    print(start_date)
    

# Print the parent DataFrame to a CSV file
parent_df.to_csv('aux_historical_data.csv', index=True)

2023-06-24 21:40:23.129211
2023-06-29 21:40:23.129211
2023-07-04 21:40:23.129211
2023-07-09 21:40:23.129211
2023-07-14 21:40:23.129211
2023-07-19 21:40:23.129211
2023-07-24 21:40:23.129211
2023-07-29 21:40:23.129211
2023-08-03 21:40:23.129211
2023-08-08 21:40:23.129211
2023-08-13 21:40:23.129211
2023-08-18 21:40:23.129211
2023-08-23 21:40:23.129211
2023-08-28 21:40:23.129211
2023-09-02 21:40:23.129211
2023-09-07 21:40:23.129211
2023-09-12 21:40:23.129211
2023-09-17 21:40:23.129211
2023-09-22 21:40:23.129211
2023-09-27 21:40:23.129211
2023-10-02 21:40:23.129211
2023-10-07 21:40:23.129211
2023-10-12 21:40:23.129211
2023-10-17 21:40:23.129211
2023-10-22 21:40:23.129211
2023-10-27 21:40:23.129211
2023-11-01 21:40:23.129211
2023-11-06 21:40:23.129211
2023-11-11 21:40:23.129211
2023-11-16 21:40:23.129211
2023-11-21 21:40:23.129211
2023-11-26 21:40:23.129211
2023-12-01 21:40:23.129211
2023-12-06 21:40:23.129211
2023-12-11 21:40:23.129211
2023-12-16 21:40:23.129211
2023-12-21 21:40:23.129211
2

In [5]:
print(parent_df)

                           open        high         low       close      macd  \
datetime                                                                        
2023-06-24 00:00:00  1932.19995  1932.30005  1930.20996  1930.69995  -0.11952   
2023-06-23 23:59:00  1933.68994     1933.88  1932.31995  1932.31995   0.17478   
2023-06-23 23:58:00  1934.34998  1934.34998  1933.54004     1933.63   0.38292   
2023-06-23 23:57:00  1935.45996  1935.68005  1934.33997  1934.33997   0.50962   
2023-06-23 23:56:00  1936.06006  1936.79004  1935.44995  1935.44995   0.59164   
...                         ...         ...         ...         ...       ...   
2024-08-07 00:04:00  2397.23999  2400.68994  2397.23999   2399.6001   0.69845   
2024-08-07 00:03:00  2397.59009  2397.73999  2396.97998   2397.1001    0.4176   
2024-08-07 00:02:00  2396.72998  2397.31006  2396.27002  2397.30005   0.30762   
2024-08-07 00:01:00  2396.44995  2397.53003  2395.88989  2396.56006    0.1424   
2024-08-07 00:00:00  2395.36

In [6]:
parent_df.to_csv('aux_historical_data.csv', index=True)
